In [1]:
# -*- coding: utf-8 -*-

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy

In [3]:
import os
import sys

## Handle the CV files

In [4]:
cv_path = os.path.join(os.getcwd(), 'data', 'sharepoint', 'cv')
cvs = os.listdir(cv_path)

In [5]:
def get_cv(i):
    return os.path.join(cv_path, cvs[i])

In [6]:
example = get_cv(0)
example

'C:\\Users\\tollef\\Documents\\Git\\nlp-intro\\data\\sharepoint\\cv\\albert danielsen.txt'

## Fetch and cleandata

In [7]:
# prepare spacy nlp object
spacy_model = 'nb_core_news_lg'
nlp = spacy.load(spacy_model, disable = ['parser', 'tagger', 'ner'])
nlp.add_pipe('sentencizer')

In [8]:
data = None
with open(example, 'r', encoding='utf-8') as f:
    data = f.readlines()

In [9]:
def get_tokens(text):
    skip_pattern = '\r\n \n\n \n\n\n!"-#$%&()--.*+,-./:;<=>?@[\\]^_`{|}~\t\n\r '
    return [token for token in text if token not in skip_pattern]

def clean_sent(sent):
    illegal = ['\n', '\t', '.', ':']
    def valid(tok):
        return tok not in illegal
    
    return ' '.join([tok for tok in sent.split(' ') if valid(tok)])
    

# combine all data, use the spacy sentencizer
def get_sentences(data):
    sentences = [clean_sent(sent).strip() for sent in data]
    return '. '.join(sentences).strip()

In [10]:
text = get_sentences(data)
print(text)

Albert er en svilingeniør innen kybernetikk og robotikk, utdannet ved NTNU, med spesialisering innen embedded systems. Han har opparbeidet seg bred erfaring innen programvareutvikling fra sine prosjekter hos tidligere arbeidsgivere og som konsulent.. . Nylig har Albert jobbet i et prosjekt for Altera Infrastructure, et stort skipsrederi, med mål om å digitalisere arbeidsflyten ved modifikasjoner på skip i drift. Hans bidrag i dette prosjektet består i front-end utvikling av en responsiv og brukervennlig løsning i Vue, samt back-end utvikling i .NET Core i et domain driven design-mønster. Gjennomførelsen av prosjektet ble gjort ved bruk av Azure DevOps og agile arbeidsmetoder.. . Tidligere har Albert bidratt på et stort prosjekt for Sjøfartsdirektoratet. Prosjektet, titulert APS (Automatisering av Personell Sertifikat), tok sikte på å automatisere registrering og utdeling av sertifikater til sjøfolk som opererer i norske farvann. Løsningen ble utviklet med React, Azure Functions og mess

In [11]:
spacy_doc = nlp(text)
MIN_LEN = 25
sentences = [s.text.replace('.','').strip() for s in spacy_doc.sents if len(s) > MIN_LEN]
sentences

['Nylig har Albert jobbet i et prosjekt for Altera Infrastructure, et stort skipsrederi, med mål om å digitalisere arbeidsflyten ved modifikasjoner på skip i drift',
 'Hans bidrag i dette prosjektet består i front-end utvikling av en responsiv og brukervennlig løsning i Vue, samt back-end utvikling i NET Core i et domain driven design-mønster',
 'Prosjektet, titulert APS (Automatisering av Personell Sertifikat), tok sikte på å automatisere registrering og utdeling av sertifikater til sjøfolk som opererer i norske farvann',
 'Alberts kompetanse inkluderer IoT-utvikling, programmering av mikrokontrollere og design av hardware og kretskort, kompetanse han har styrket ved praktisk gjennomføring av prosjekt og deltidsjobb for SINTEF, ved å utvikle hardware og software for et maskinelt kalibreringsverkøy for fjærstivhet og en modulær robotisk gripeenhet',
 'Teknologier og verktøy Albert behersker og liker å arbeide med er bla: Azure DevOps, NET, Git, Vue, React, JavaScript, CSS, HTML, Power 